In [2]:
!python3 -m pip install jmespath

In [83]:
!python3 -m pip install pandas



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip


In [2]:
import json
import os
import sys

#### Desired data pairs and metadata

In [3]:
import jmespath
import json
import os
import sys

# Load the JSON file
def load_json_file(file_path):
    if not os.path.exists(file_path):
        print(f"File {file_path} does not exist.")
        sys.exit(1)

    with open(file_path, 'r') as file:
        try:
            data = json.load(file)
            return data
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON from file {file_path}: {e}")
            sys.exit(1)

# Parse a single Instagram post (items[0]) using jmespath
def parse_post_json(data: dict) -> dict:
    return jmespath.search(
        """{
            shortcode: code,
            caption: caption.text,
            head_image_url: image_versions2.candidates[0].url,
            audio_type: music_metadata.audio_type,
            music_title: music_metadata.music_info.music_asset_info.title,
            music_artist: music_metadata.music_info.music_asset_info.display_artist
        }""",
        data,
    )

# Extract all parsed posts from a combined user-post JSON
def extract_all_posts(json_data: dict):
    parsed_posts = []

    for username, posts_dict in json_data.items():
        for post_url, post_data in posts_dict.items():
            items = post_data.get("items", [])
            if items:  # Only if not empty
                parsed = parse_post_json(items[0])
                if parsed:
                    parsed["username"] = username  # Add source username
                    parsed_posts.append(parsed)

    return parsed_posts


In [5]:
# Load file
raw_json_data = load_json_file("json/all_instagram_data.json")

# Parse all valid posts
parsed_posts = extract_all_posts(raw_json_data)

In [6]:
import pandas as pd

df = pd.DataFrame(parsed_posts)
len(df)

991

In [7]:
# for all "None" on audio type, remove them
df = df[df['audio_type'] == 'licensed_music']

# remove id, caption, audio type, 
df = df.drop(columns=['username', 'caption', 'audio_type'])

df

,shortcode,head_image_url,music_title,music_artist
81,DHqyEKlx45m,https://scontent-lax3-1.cdninstagram.com/v/t51...,Call Me When You Break Up,"Selena Gomez, benny blanco, Gracie Abrams"
92,DHKpuJEtril,https://scontent-lax3-2.cdninstagram.com/v/t51...,Sunset Blvd,"Selena Gomez, benny blanco"
104,C6bGvvzvQma,https://scontent-lax3-1.cdninstagram.com/v/t51...,Perfect,Ed Sheeran
107,C0vRPnvoIAa,https://scontent-lax3-1.cdninstagram.com/v/t51...,Those Eyes,New West
165,C_gUrFoSjoi,https://scontent-lax3-1.cdninstagram.com/v/t51...,I've Been In Love (feat. Channel Tres),Jungle
...,...,...,...,...
971,DDqSwt_z8nJ,https://scontent-lax3-1.cdninstagram.com/v/t51...,Be Be Your Love,Rachael Yamagata
972,DCyN-bJTPBy,https://scontent-lax3-1.cdninstagram.com/v/t51...,Choose You,Elmiene
975,DCWjVQ5Tt71,https://scontent-lax3-2.cdninstagram.com/v/t51...,Wings,Mot
987,DAgUVg_TFDp,https://scontent-lax3-2.cdninstagram.com/v/t51...,hotline (edit),Billie Eilish


In [ ]:
df_old = pd.read_csv("csv/all_instagram_data.csv")

#concat
df = pd.concat([df_old, df], ignore_index=True)
# remove duplicates
df = df.drop_duplicates()
# save to csv
df.to_csv("csv/all_instagram_data.csv", index=False)